In [44]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations

#todo: fix smooth rainfall missing entries 
start = 366
end = 911

class Source: 
    def __init__(self, name, data, is_daily, s_type, capex, opex, rententate):

        self.name = name
        self.data = data      
        self.is_daily = is_daily
        self.s_type = s_type
        self.capex = capex
        self.opex = opex
        self.rententate = rententate
        
        self.reservoir_size = None
        self.reservoir_history = None    
        
        
    
class Pool:
    def __init__(self, source_list):
            
        self.source_list = source_list

        self.names = ''.join(s.name for s in self.source_list)

        self.reservoir_size = None
        self.reservoir_history = None
        self.capex = None
        self.opex = None
        self.rententate = None
        self.amounts = None
        
        
    def get_data(self):
        if self.amounts == None:
            slist = self.source_list

            s0 = slist.pop()                  
            self.amounts = s0.data

            while slist:
                s = slist.pop()
                self.amounts += s.data

        return self.amounts
        
class SP2X:
    def __init__(self, gen_cap, water_2_x, retention_fac, cap_fac, buffer): 

        self.gen_cap = gen_cap
        self.water_2_x = water_2_x
        self.retention_fac = retention_fac
        self.cap_fac = cap_fac
        self.buffer = buffer         
        
        self.water_need = None
        #TODO: fix hardcode
        self.water_need = self.p2x_need()

    def p2x_need(self):
        need = []     
        for _ in range(100000):
            mWh = 24 * (self.cap_fac / 100) * self.gen_cap;
            need.append(self.water_2_x * mWh / self.retention_fac)
        return need

    def analyse(self, s):
        
        data = s.get_data()
        balance = self.water_balance(data)
        volume = np.zeros(len(data))
        capacity = 0 
        volume[start-1] = capacity 
        reservoirRatio = []
        isSatisfied = False

        while not (isSatisfied or capacity == -1):
            capacity += 100
            for x in range(start, len(data)):

                volume[x] = volume[x-1] + balance[x]

                #reservoir cant contain more than full capacity
                if volume[x] > capacity: 
                    volume[x] = capacity

                if (volume[x] / capacity) * 100 < self.buffer: 
                    break

                if x == end: 
                    isSatisfied = True 

                if capacity > 10000: 
                    capacity = -1

        print(capacity)
            

    def water_balance(self, water):
        balance = []
        #TODO: change from zero to none (or equivilent), after fixing missing values as zero 
        for x in range(len(water)):
            if water[x] == 0:
                balance.append(0)
            else:
                balance.append(water[x] - self.water_need[x])
        return balance



        #day = list(range(len(r.amounts)))
            




In [51]:
#data = pd.read_csv(r'C:\Users\Lucas\Desktop\Arbejde\sekundavand\Data\preproccessing\monthVols\MonthData.csv', sep=';')
data = pd.read_excel(r'C:\Users\Lucas\Desktop\Arbejde\sekundavand\Data\proccessedData.xlsx')

            #name,        data,              is_daily,  s_type,  capex, opex, rententate
s1 = Source("Bruns Nord", data["P1DailyVol"]*0.45, True, "floodwater", 0, 0, [])
s2 = Source("Normark", data["P1DailyVol"], True, "floodwater", 0, 0, [])
s3 = Source("Sandholm", data["P3DailyVol"]*0.45, True, "floodwater", 0, 0, [])
s4 = Source("Hovsøre", data["P3DailyVol"], True, "floodwater", 0, 0, [])
s5 = Source("Plet Enge", data["P3DailyVol"], True, "floodwater", 0, 0, [])

#    gen_cap, water_2_x, retention_fac, cap_fac, buffer         
p2x = SP2X(500, 0.2, 0.65, 66, 30)


p1 = Pool([s1, s2])

p2x.analyse(p1)



700


In [ ]:

s1.info()



In [ ]:
s1_s2 = Pool([s1, s2])




In [ ]:

p2x.analyse(s1_s2)


In [ ]:
s1_s2.info()